In [1]:
import numpy as np

class LFSR:
    
    def __init__(self, seed=0x0123456789bdefa2154acbd55a468daf5, randomSeed=False):
        self.seed = seed
        self.reg = np.zeros(130, dtype = int)
        self.randomSeed = randomSeed
        
        if self.randomSeed:
            self.seed = np.random.randint(2, size=130)
            self.seed = np.flip(self.seed)
            self.reg = self.seed.copy()
        else:
            self.seed = bin(seed)[2:].zfill(130)
            if len(self.seed) > 130:
                self.seed = self.seed[len(self.seed) - 130:]
            for i in range(130):
                self.reg[i] = int(self.seed[i])
            
            self.reg = np.flip(self.reg)
            self.seed = self.reg.copy()
            
    def reset(self):
        self.reg = self.seed.copy()
        
    def getRandom(self):
        result =  [self.reg[0:32].tolist(), self.reg[32:64].tolist(), self.reg[64:96].tolist(), self.reg[96:128].tolist()]
        self.update()
        return result
    
    
    # For debugging
    def printSeed(self):
        print("LFSR Seed is :", np.flip(self.seed))
        
    # For debugging
    def printReg(self, showHex = True):
        if showHex:
            result = ""
            result += hex(int(str(self.reg[129]) + str(self.reg[128]), 2))[2:]
            for i in range(127, 0, -4):
                fourDigit = ""
                for j in range(0, 4, 1):
                    fourDigit += str(self.reg[i - j])
                    
                result += hex(int(fourDigit, 2))[2:]
                
            print(result)
        else:
            print(np.flip(self.reg))
        
        
    def update(self):
        nextReg = np.zeros(130, dtype = int)
        for i in range(0, 120):
            nextReg[i] = self.reg[i + 7] ^ self.reg[i + 10]
            
        for i in range(120, 130):
            nextReg[i] = self.reg[i - 120]
    
        self.reg = nextReg

In [2]:
class hat_mul:
    def __init__(self):
        self.z = np.zeros(16, dtype = int)
        self.z1 = np.zeros(16, dtype = int)
        self.z2 = np.zeros(16, dtype = int)
        self.a = 0
        self.b = 0
    
    def bit_xnor(self, a, b):
        if a == b:
            return 1
        else:
            return 0
        
    def update(self, in_32):
        self.a = (in_32[20] | in_32[21] | in_32[22]) & in_32[23]
        self.b = in_32[24] & in_32[25] & in_32[26]
        nz = np.zeros(16, dtype = int)
        nz1 = np.zeros(16, dtype = int)
        nz2 = np.zeros(16, dtype = int)
        
        
        # update z
        nz[0:13] = in_32[0:13]
        nz[15] = in_32[15]
        if self.a:
            nz[13] = in_32[15]
            nz[14] = in_32[15]
        else:
            nz[13:15] = in_32[13:15]
        
        # update z1
        nz1[0:14] = self.z[0:14]
        nz1[15] = self.z[15]
        if self.b:
            nz1[14] = self.z[14]
        else:
            nz1[14] = self.z[15]
            
        #update z2
        nz2[0:13] = self.z1[0:13]
        nz2[15] = self.z1[15]
        if (self.z1[14] ^ self.z1[15]) & (self.bit_xnor(self.z1[13], self.z1[14])):
            if (in_32[27] ^ self.z1[15]) & (self.bit_xnor(in_32[28], in_32[27])):
                nz2[13:15] = in_32[29:31]
            else:
                nz2[13:15] = in_32[27:29]
        else:
            nz2[13:15] = self.z1[13:15]
            
        self.z = nz
        self.z1 = nz1
        self.z2 = nz2
        
        return self.z2
        
    def reset(self):
        self.z = np.zeros(16, dtype = int)
        self.z1 = np.zeros(16, dtype = int)
        self.z2 = np.zeros(16, dtype = int)
        self.a = 0
        self.b = 0
        

In [3]:
class adder_block:
    
    def __init__(self):
        self.s00 = "00000000000000000"
        self.s01 = "00000000000000000"
        self.s10 = "00000000000000000"
        
    def reset(self):
        self.s00 = "00000000000000000"
        self.s01 = "00000000000000000"
        self.s10 = "00000000000000000"
        
    def update(self, in0, in1, in2, in3):
        self.s10 = self.binaryAdd(self.s00[1:] + self.s00[16], self.s01[1:] + self.s01[16])
        in0 = self.convertToStr(in0) + str(in0[15])
        in1 = self.convertToStr(in1) + str(in1[15])
        in2 = self.convertToStr(in2) + str(in2[15])
        in3 = self.convertToStr(in3) + str(in3[15])
        self.s00 = self.binaryAdd(in0, in1)
        self.s01 = self.binaryAdd(in2, in3)
        return self.convert2scomp(self.s10[1:]) / 4096
    
    def convertToStr(self, num):  
        result = ""
        for i in range(0, len(num)):
            result += str(num[i])
    
        return result
    
    def binaryAdd(self, a, b):
        carry = 0
        result = ""
        for i in range(0, len(a)):
            bit1 = int(a[i])
            bit2 = int(b[i])
            s = bit1 ^ bit2 ^ carry
            nextCarry = (bit1 and bit2) or (bit1 and carry) or (bit2 and carry)
            
            result += str(s)
            carry = nextCarry
            
        return result
    
    def convert2scomp(self, num):
        if num[-1] == "1":
            newNum = ""
            for i in range(16):
                if num[i] == "1":
                    newNum += "0"
                else:
                    newNum += "1"
            num = newNum
            num = self.binaryAdd(num, "1000000000000000")
            return -1 * int(num[::-1], 2)
        else:
            return int(num[::-1], 2)

In [4]:
class Multihat:
    
    def __init__(self):
        self.myLFSR = LFSR()
        self.h1 = hat_mul()
        self.h2 = hat_mul()
        self.h3 = hat_mul()
        self.h4 = hat_mul()
        self.ab = adder_block()
        
    def get(self):
        in_32 = self.myLFSR.getRandom()
        in0 = self.h1.update(in_32[0])
        in1 = self.h2.update(in_32[1])
        in2 = self.h3.update(in_32[2])
        in3 = self.h4.update(in_32[3])
        return self.ab.update(in0, in1, in2, in3)
    
    def debug_convertArrayToHex(self, ls, message):
        result = ""
        for i in range(15, 0, -4):
            temp = ""
            for j in range(0, 4, 1):
                temp += str(ls[i - j])
            temp = hex(int(temp, 2))[2:].zfill(1)
            result = result + temp
            
        print(message + result)

In [5]:
rand = Multihat()
for i in range(100):
    print(rand.get())

0.0
0.0
0.0
-0.361328125
-0.78369140625
-0.928466796875
-0.667724609375
0.501708984375
0.317138671875
-0.79638671875
0.709716796875
1.208740234375
-0.539306640625
-0.45458984375
0.35693359375
-0.472412109375
-0.156005859375
-1.22607421875
-1.22607421875
0.029052734375
0.072998046875
1.672607421875
-0.03515625
-0.80908203125
-0.49658203125
0.50146484375
-0.105712890625
-0.419921875
-0.8193359375
-1.7373046875
0.25
1.444580078125
-2.4189453125
-0.27001953125
-0.37451171875
-0.336181640625
-0.38134765625
-0.750244140625
1.260986328125
0.53076171875
-0.59326171875
-0.718017578125
0.492431640625
-1.201904296875
-1.25
-0.540283203125
0.7197265625
0.621337890625
1.361083984375
1.52099609375
0.469970703125
0.170166015625
0.66015625
0.04541015625
-0.224853515625
-1.201171875
0.75634765625
0.112548828125
0.941650390625
-1.262451171875
0.384033203125
0.81591796875
-0.6884765625
0.012939453125
-0.603759765625
0.77783203125
0.07958984375
1.68798828125
-0.962890625
-1.00244140625
-0.797119140625
1.1